# Importing modules

In [ ]:
import numpy as np
import os
import gdal
import ogr
import osr
import gdalnumeric
import gdalconst
from osgeo.gdalconst import GA_ReadOnly
import pandas as pd
import sys

# Defining base functions

In [ ]:
#all Functions

def find_band_number(dataset, variable):
    '''
    Finds the band number and level inside the GRIB file, given the variable
    '''
    i_list = []
    level_list  =[]
    for i in range(1,dataset.RasterCount + 1):
        band = dataset.GetRasterBand(i)
        metadata = band.GetMetadata()
        band_level = metadata['GRIB_SHORT_NAME']
        band_variable = metadata['GRIB_ELEMENT']
        level = band_level[-4:]
        if (variable == band_variable) and (level == 'ISBL'):
            i_list = i_list + [i]
            level = band_level[0:-5]
            level_list = level_list + [level]
            #return i
    return i_list, level_list #retun a list with the number in the band of the variable and its ISBL level
    
def get_Timestamp(Y, M, D, hour, forc):
    '''
    Take as input the YMD (year, month and day in the format YYYYMMDD), hour (format HH) and forc 
    (format HHH - hours betweent forecast and simulation). The output is a pandas Timestamp. 
    '''
    hour = hour[0:2]
    hour = int(hour)
    forc = int(forc)
    hour = hour + forc
    if hour > 23:
        
        D = int(D) + int(hour/24)
        if D < 10:
            D = '0' + str(D)
        else:
            D = str(D)
        hour = hour%24
    if hour < 10:
        hour = '0'+str(hour)
    else:
        hour = str(hour)
    time = Y + M + D + ' ' + hour
    
    try:
        timestamp =  pd.Timestamp(time)
    
    except ValueError:
        try:
            M = int(M) + 1
            D = '01'
            if M < 10:
                M = '0' + str(M)
            else:
                M = str(M)
            time = Y + M + D + ' ' + hour
            timestamp =  pd.Timestamp(time)
        except:
            Y = int(Y) +1
            Y = str(Y)
            M = '01'
            timestamp =  pd.Timestamp(time)
    return timestamp

# Defining variables to the conversion

In [ ]:
# to read the data
year = '2018'
monthh = ['01','02','03','04','05','06','07','08','09','10','11','12']
dayy = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16',
        '17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
hour = ['0000','0600','1200','1800']
forc = ['006']
data_type = 'gfs_3'
#directory of the files
directory = 'data_Grib/'

In [ ]:
# Defining constants
Nb = 1000 #number of files
Nx = 181 #number of latitudes
Ny = 360 #number of longitudes
# to save only a single level in each matrix
level_set = '20000'

# Converting from grib to Matrix

In [ ]:
#initializing matrix and variables
Matrix_u = np.zeros([Nx,Ny,Nb])
Matrix_v = np.zeros([Nx,Ny,Nb])
Matrix_T = np.zeros([Nx,Ny,Nb])
delta_hours = np.zeros([Nb])
delta_hours[0] = 0
vec_time = []
count = 0

#looping between the forescat data
for month in monthh:
    YM = year + month 
    print('\n',YM)
    for day in dayy:
        D = day
        print(D,end = '|')
        for i1 in range(len(hour)):
            for j1 in range(len(forc)):
                hour1 = hour[i1]
                forc1 = forc[j1]
                file_name = data_type +  '_' +  YM + D + '_'   + hour[i1] + '_' + forc[j1] + '.grb2'
                data_file = directory + file_name
                # verify if file exists in the directory
                if os.path.isfile(data_file):
                    # gets the timestamp
                    time = get_Timestamp(year, month, D, hour1, forc1)
                    vec_time += [time]
                    
                    #The parsing of the Grib is based in: 
                        #http://geoexamples.blogspot.com/2013/05/drawing-wind-barbs-gdal-python.html
                        
                    # importing data from Grib file
                    dataset = gdal.Open(data_file, GA_ReadOnly )
                    #get variables
                    u_band_id,u_band_level = find_band_number(dataset, 'UGRD') #Wind Wx
                    v_band_id,v_band_level = find_band_number(dataset, 'VGRD') #Wind Wy
                    T_band_id,T_band_level = find_band_number(dataset, 'TMP') #Temperature

                    #print(u_band_id)
                    i = 0
                    level = u_band_level
                    #level_set = '20000'
                    while level[i] != level_set:
                        i += 1
                        #print(i)
                    band_u = dataset.GetRasterBand(u_band_id[i])
                    band_v = dataset.GetRasterBand(v_band_id[i])
                    band_T = dataset.GetRasterBand(T_band_id[i])
                    level = u_band_level[i]
                    
                    #print(level)
                    geo = dataset.GetGeoTransform()
                    xsize = band_u.XSize
                    ysize = band_u.YSize

                    
                    Matrix_u [:,:,count] = band_u.ReadAsArray(0, 0, xsize, ysize)
                    Matrix_v [:,:,count] = band_v.ReadAsArray(0, 0, xsize, ysize)
                    Matrix_T [:,:,count] = band_T.ReadAsArray(0, 0, xsize, ysize)
                    count += 1


# generate the vector which contains the number of hours between each forecast                        
for i in range(1,count):
    delta_hours[i] = ((vec_time[i]- vec_time[i-1]).total_seconds()/3600)
    
# in case the number of files, Nb, was specified greater than the reality it corrects the vector    
Matrix_u = Matrix_u [:,:,:count]
Matrix_v = Matrix_v [:,:,:count]
Matrix_T = Matrix_T [:,:,:count]
delta_hours = delta_hours[:count]
vec_time = vec_time[:count]

# Save the matrix

In [ ]:
# chooses the name of the file
file_save = (YM + 'Wind_UVT-level_' + level_set + '-1Forc_' + forc[0])
print('\n',file_save)

# directory to save the data
directory = 'data_matrix/'
filename = directory + file_save + '.npz'
# saving the data
np.savez(filename, Wx = Matrix_u, Wy = Matrix_v, T = Matrix_T, delta_hours = delta_hours, vec_time = vec_time)
# complete data name
print(filename)